In [268]:
import numpy as np
from numba import njit
import numba as nb
import pickle
from numba.typed import List

from scipy.spatial.distance import cdist, pdist

In [273]:
var = pickle.load(open('var1.pickle', 'rb'))

In [274]:
data = var['Data2[k,:]'] 
Box = var ['BOX']
BOX_miu = var ['BOX_miu']
BOX_S = var ['BOX_S']
NB = var ['NB']
intervel1 = var ['interval1']
intervel2 = var ['interval2']

print(type(data))
print(type(Box))
print(type(BOX_miu))
print(type(BOX_S))
print(type(NB))
print(type(intervel1))
print(type(intervel2))

<class 'numpy.matrix'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'int'>
<class 'numpy.float64'>
<class 'numpy.float64'>


In [275]:
print(Box)
print(BOX_miu)
print(BOX_S)
print(NB)
print(intervel1)
print(intervel2)

[]
[]
[]
277
1.391359885272717
0.19814776281719967


In [276]:
def Chessboard_online_division_std(data,Box,BOX_miu,BOX_S,NB,intervel1,intervel2):
    distance = np.zeros((NB,2))
    COUNT = 0
    SQ = []

    for i in range(NB):


        distance[i,0] = pdist([list(BOX_miu[i]), data.tolist()[0]],'euclidean')

        distance[i,1] = np.sqrt(pdist([list(BOX_miu[i]), data.tolist()[0]],'cosine'))
        
        if distance[i,0] < intervel1 and distance[i,1] < intervel2:
            COUNT += 1
            SQ.append(i)

    if COUNT == 0:
        Box_new = np.concatenate((Box, np.array(data)))
        NB_new = NB+1
        BOX_S_new = np.concatenate((BOX_S, np.array([1])))
        #BOX_S_new = np.array(BOX_S)
        BOX_miu_new = np.concatenate((BOX_miu, np.array(data)))
    if COUNT>=1:
        DIS = np.zeros((COUNT,1))
        for j in range(COUNT):
            DIS[j] = distance[SQ[j],0] + distance[SQ[j],1]
        a = np.amin(DIS)
        b = int(np.where(DIS == a)[0])
        Box_new = Box
        NB_new = NB
        BOX_S_new = np.array(BOX_S)
        BOX_miu_new = np.array(BOX_miu)
        BOX_S_new[SQ[b]] = BOX_S[SQ[b]] + 1
        BOX_miu_new[SQ[b]] = BOX_S[SQ[b]]/BOX_S_new[SQ[b]]*BOX_miu[SQ[b]]+data/BOX_S_new[SQ[b]]
    
            
    
    return Box_new,BOX_miu_new,BOX_S_new,NB_new

In [277]:
%%time
Box_new_std, BOX_miu_new_std, BOX_S_new_std, NB_new_std = Chessboard_online_division_std(data,Box,BOX_miu,BOX_S,NB,intervel1,intervel2)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [278]:
@njit
def Chessboard_online_division_njit(data,Box,BOX_miu,BOX_S,NB,intervel1,intervel2):
    distance = np.zeros((NB,2))
    COUNT = 0
    SQ = []
    
    
    _, W = data.shape
    for i in range(NB):
        # pdist euclidean
        # distance[i,0] = pdist([list(BOX_miu[i]), data.tolist()[0]],'euclidean')
            
        aux = 0
        for iii in range(W):
            aux += (BOX_miu[i, iii] - data[0, iii])**2 
                
        distance[i,0] = aux**.5
                
        # pdist cosine
        # distance[i,1] = np.sqrt(pdist([list(BOX_miu[i]), data.tolist()[0]],'cosine'))
            
        num = 0
        den1 = 0
        den2 = 0
        for iii in range(W):
            num += BOX_miu[i,iii]*data[0, iii]
            den1 += BOX_miu[i,iii]**2
            den2 += data[0, iii]**2
            
        d2 = (1 - num/(den1**.5 * den2**.5) )
        if d2 < 0:
            distance[i,1] = 0
        else:
            distance[i,1] = d2**.5
                
        
        if distance[i,0] < intervel1 and distance[i,1] < intervel2:
            COUNT += 1
            SQ.append(i)

            
    Box_new = List(Box)
    BOX_S_new = List(BOX_S)
    BOX_miu_new = List(BOX_miu)
    
    if COUNT == 0:
        Box_new.append(List(data)[0])
        NB_new = NB+1
        BOX_S_new.append(nb.int32(1))
        BOX_miu_new.append(List(data)[0])
        
    if COUNT>=1:
        DIS = np.zeros((COUNT,1))
        for j in range(COUNT):
            DIS[j] = distance[SQ[j],0] + distance[SQ[j],1]
        
        mini = DIS[0]
        for ii in range(1,len(DIS)):
            if DIS[ii] < mini:
                mini = DIS[ii]
                b = ii

        
        NB_new = NB
        BOX_S_new[SQ[b]] = BOX_S[SQ[b]] + 1
        for i in range(W):
            BOX_miu_new[SQ[b]][i] = BOX_S[SQ[b]]/BOX_S_new[SQ[b]]*BOX_miu[SQ[b]][i]+data[0,i]/BOX_S_new[SQ[b]]
    
    
    #Box_new = np.asarray(Box_new)
    #BOX_miu_new = np.asarray(BOX_miu_new)
    #BOX_S_new = np.asarray(BOX_S_new)
    return Box_new,BOX_miu_new,BOX_S_new,NB_new

In [279]:
%%time
Box_new_njit, BOX_miu_new_njit, BOX_S_new_njit, NB_new_njit = Chessboard_online_division_njit(np.array(data),Box,BOX_miu,BOX_S,NB,intervel1,intervel2)

Box_new_njit = np.asarray(Box_new_njit)
BOX_miu_new_njit = np.asarray(BOX_miu_new_njit)
BOX_S_new_njit = np.asarray(BOX_S_new_njit)

ZeroDivisionError: division by zero

In [280]:
print("----- Box_new -----")
print(sum(sum(Box_new_std - Box_new_njit)))
print("STD Type: ", type(Box_new_std))
print("NJIT Type: ", type(Box_new_njit))

print("----- Box_miu_new -----")
print(sum(sum(BOX_miu_new_std - BOX_miu_new_njit)))
print("STD Type: ", type(BOX_miu_new_std))
print("NJIT Type: ", type(BOX_miu_new_njit))

print("----- Box_S_new -----")
print(sum(BOX_S_new_std - BOX_S_new_njit))
print("STD Type: ", type(BOX_S_new_std))
print("NJIT Type: ", type(BOX_S_new_njit))

print("----- NB_new -----")
print(NB_new_std - NB_new_njit)
print("STD Type: ", type(NB_new_std))
print("NJIT Type: ", type(NB_new_njit))

----- Box_new -----


ValueError: operands could not be broadcast together with shapes (277,8) (437,8) 